In [34]:
update_mode = 'daily'

In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import sys
import hopsworks
from config import hopsworks_api_key
import great_expectations as ge
from config import Airport_Coordinates, Arlanda

In [29]:
def fetch_smhi_weather(latitude, longitude):
    url = f"https://opendata-download-metfcst.smhi.se/api/category/pmp3g/version/2/geotype/point/lon/{longitude}/lat/{latitude}/data.json"

    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        data = response.json()

        forecasts = []

        for ts in data['timeSeries']:
            valid_time = datetime.fromisoformat(ts['validTime'].replace('Z', '+00:00'))
            params = {p['name']: p['values'][0] for p in ts['parameters']}

            forecast = {
                'timestamp': valid_time.strftime('%Y-%m-%d %H:%M:%S'),
                'temperature': params.get('t', None), # In Celsius
                'wind_speed': params.get('ws', None), # In m/s
                'wind_direction': params.get('wd', None), # In Degrees
                'humidity': params.get('r', None), # In %
                'pressure': params.get('msl', None), # In hPa
                'precipitation': params.get('pcat', None), # Category
                'visibility': params.get('vis', None), # In km
                'cloud_cover': params.get('tcc_mean', None) # In Octas/Oktas
            }

            forecasts.append(forecast)

        return forecasts

    except Exception as e:
        print(f"Error fetching weather forecast at Arlanda: {e}")
        return []

In [30]:
def create_weather_condition(precipitation, visibility, wind_speed):
    if precipitation in [1,2]:
        return 'snow'
    elif precipitation == 3:
        if wind_speed > 10:
            return 'rain_windy'
        else:
            return 'rain'
    elif visibility and visibility < 1:
        return 'fog'
    elif wind_speed and wind_speed > 15:
        return 'windy'
    else:
        return 'clear'

In [31]:
def fetch_arlanda_weather():
    coords = Airport_Coordinates[Arlanda]
    forecasts = fetch_smhi_weather(coords['lat'], coords['long'])
    
    for f in forecasts:
        f['airport_code'] = Arlanda
        f['weather_condition'] = create_weather_condition(
            f.get('precipitation'), 
            f.get('visibility'), 
            f.get('wind_speed')
        )
        f['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return pd.DataFrame(forecasts)

In [32]:
def update_weather_features():
    df_weather = fetch_arlanda_weather()

    project = hopsworks.login(api_key_value=hopsworks_api_key, host="eu-west.cloud.hopsworks.ai")
    fs = project.get_feature_store()

    weather_features_fg = fs.get_or_create_feature_group(
        name='weather_features',
        description="Weather Features at Arlanda Airport",
        version=1,
        primary_key = ["airport_code", "timestamp"]
    )

    weather_features_fg.insert(df_weather, overwrite=False)

    weather_features_fg.update_feature_description("temperature", "In Celsius")
    weather_features_fg.update_feature_description("wind_speed", "In metres/second")
    weather_features_fg.update_feature_description("wind_direction", "In Degrees")
    weather_features_fg.update_feature_description("humidity", "In Percentages (%)")
    weather_features_fg.update_feature_description("pressure", "In hPa (HectoPascals)")
    weather_features_fg.update_feature_description("precipitation", "Categories where 1-Snow, 2-Mixed Snow, 3-Rain, 4-Drizzle")
    weather_features_fg.update_feature_description("visibility", "In kilometres")
    weather_features_fg.update_feature_description("cloud_cover", "In Octas/Oktas")

    return df_weather
    

## Uploading Created Dataframe on Hopsworks

In [33]:
update_weather_features()

2026-01-03 00:37:49,655 INFO: Closing external client and cleaning up certificates.
2026-01-03 00:37:49,666 INFO: Connection closed.
2026-01-03 00:37:49,673 INFO: Initializing external client
2026-01-03 00:37:49,673 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-03 00:37:50,555 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3207
Feature Group created successfully, explore it at 
https://eu-west.cloud.hopsworks.ai:443/p/3207/fs/3151/fg/3285


Uploading Dataframe: 100.00% |███████████████████████████████████████████████████████████████████████████████████████| Rows 78/78 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3207/jobs/named/weather_features_1_offline_fg_materialization/executions


,timestamp,temperature,wind_speed,wind_direction,humidity,pressure,precipitation,visibility,cloud_cover,airport_code,weather_condition,created_at
0,2026-01-03 00:00:00,-2.9,7.0,22,87,982.8,1,2.3,8,ARN,snow,2026-01-03 00:37:49
1,2026-01-03 01:00:00,-2.8,7.2,26,86,983.0,1,1.8,8,ARN,snow,2026-01-03 00:37:49
2,2026-01-03 02:00:00,-2.7,7.3,29,86,983.2,1,2.8,8,ARN,snow,2026-01-03 00:37:49
3,2026-01-03 03:00:00,-2.7,7.7,26,85,983.2,1,1.9,8,ARN,snow,2026-01-03 00:37:49
4,2026-01-03 04:00:00,-2.7,8.1,23,84,983.2,1,3.0,8,ARN,snow,2026-01-03 00:37:49
...,...,...,...,...,...,...,...,...,...,...,...,...
73,2026-01-10 12:00:00,-5.8,4.2,51,84,1006.3,1,37.1,7,ARN,snow,2026-01-03 00:37:49
74,2026-01-11 00:00:00,-8.2,4.1,10,86,1006.5,1,39.4,7,ARN,snow,2026-01-03 00:37:49
75,2026-01-11 12:00:00,-7.2,3.8,17,82,1007.2,1,39.3,7,ARN,snow,2026-01-03 00:37:49
76,2026-01-12 00:00:00,-9.0,3.6,23,86,1007.8,1,38.6,6,ARN,snow,2026-01-03 00:37:49
